In [ ]:
import os
from openai import AzureOpenAI
import json

# 1. 환경 변수 설정 (보안상 권장)
# 실제 프로젝트에서는 API 키를 코드에 직접 넣지 않고 환경 변수로 관리하는 것이 안전합니다.
# 예: os.environ["AZURE_OPENAI_API_KEY"] = "YOUR_API_KEY"
# 예: os.environ["AZURE_OPENAI_ENDPOINT"] = "YOUR_ENDPOINT"
# 예: os.environ["AZURE_OPENAI_API_VERSION"] = "2024-02-15-preview" # 또는 다른 안정적인 버전

# 만약 환경 변수를 사용하지 않는다면 아래 변수에 직접 값을 할당하세요.
azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY") # Azure 포털에서 복사한 키
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT") # Azure 포털에서 복사한 엔드포인트 URL
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-15-preview")  # 기본값
your_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT", "gpt-4o-mini")

# 2. AzureOpenAI 클라이언트 초기화
client = AzureOpenAI(
    api_key=azure_openai_api_key,
    azure_endpoint=azure_openai_endpoint,
    api_version=azure_openai_api_version
)

# 3. Chat Completion 요청 보내기
try:
    # 제안하는 LLM 프롬프트 내용 (긴 문자열로 정의)
    # 파이썬의 삼중 따옴표(```)를 사용하면 여러 줄의 문자열을 편리하게 작성할 수 있습니다.
    system_prompt_content = """
You are an AI assistant designed to extract dining preferences from user queries and format them into a structured JSON object.
Your primary goal is to **always output a JSON object** following the specified schema.
You must not provide any additional conversational text, explanations, or deviations from this JSON format.

**JSON Output Format:**

```json
{
    "인원": [Integer],
    "계절": [String, e.g., "봄", "여름", "가을", "겨울"],
    "점저": [String, e.g., "점심", "저녁"],
    "1인당비용": [Integer],
    "업종 중분류": [String, e.g., "한식", "양식", "중식", "일식", "카페/디저트", "술집", "치킨", "분식", "패스트푸드" 등 구체적인 음식/업종 카테고리],
    "구": [String, e.g., "강남구", "마포구", "중구", "종로구", "서초구", "영등포구" 등 대한민국 서울특별시 및 주요 도시의 실제 '구' 이름. 만약 '구' 정보가 명확하지 않거나 대한민국이 아닌 지역이 언급되면, 사용자에게 직접 묻거나 가장 합리적인 서울/경기권 '구'로 가정합니다.]
}
```

**Instructions for Populating the JSON Fields:**
  
* **인원 (Participants):**
    * Extract the number of people directly mentioned.
    * If not explicitly stated, infer based on common phrases (e.g., "혼자" -> 1, "둘이서" -> 2, "팀원들" -> 문맥상 파악 가능한 인원 또는 합리적인 기본값).
    * If no clear number is inferable, default to `1`.
* **계절 (Season):**
    * Always determine the current season based on the current date. (현재는 여름으로 설정).
* **점저 (Meal Time):**
    * Determine if the user is referring to "점심" (lunch) or "저녁" (dinner) based on keywords like "점심", "저녁", "회식", "퇴근 후", "아침" (이 경우 점심 또는 저녁으로 변환).
    * If ambiguous, default to "저녁".
* **1인당비용 (Cost per Person):**
    * Calculate this by dividing the total budget by the number of people.
    * If a total budget is given without the number of people, use the inferred number of people.
    * If neither total budget nor explicit cost per person is given, infer a reasonable cost per person based on the `업종 중분류` (e.g., 치킨/분식은 저렴하게, 양식/일식은 높게).
* **업종 중분류 (Cuisine/Category):**
    * Identify the most specific type of cuisine or establishment mentioned (e.g., "한식", "양식", "중식", "일식", "카페/디저트", "술집", "치킨", "피자", "분식", "패스트푸드").
    * Be flexible with synonyms and general terms.
    * If no specific type is mentioned, try to infer from the context (e.g., "밥 먹자" -> "한식"). If still unclear, default to "한식".
* **구 (District):**
    * Extract the specific '구' name (e.g., "강남구", "마포구", "중구").
    * **CRITICAL:** If the user mentions a location outside of South Korea (e.g., "미국", "중국", "뉴욕", "신주쿠구") or a clearly non-existent district, still extract what they said into the `구` field. **DO NOT** output a "service not supported" message from the LLM. The downstream application will handle this validation.
    * If no district is mentioned, assume a reasonable default based on common activity (e.g., "중구" 또는 "강남구" 등 서울의 중심지).

**Example Scenarios:**

* **Scenario 1 (Normal):**
    * **User Input:** "나는 마포구에서 회사를 다니는 직장인이야 오늘 저녁에 팀원 8명과 중식집에서 총 30만원 이내로 회식을 하려고 해"
    * **Expected Output:**
        ```json
        {
            "인원": 8,
            "계절": "여름",
            "점저": "저녁",
            "1인당비용": 37500,
            "업종 중분류": "중식",
            "구": "마포구"
        }
        ```
* **Scenario 2 (Ambiguous/Missing Info):**
    * **User Input:** "오늘 점심 뭐 먹지?"
    * **Expected Output (assuming 1인당비용은 문맥상 합리적으로 추론):**
        ```json
        {
            "인원": 1,
            "계절": "여름",
            "점저": "점심",
            "1인당비용": 10000,
            "업종 중분류": "한식",
            "구": "중구"
        }
        ```
* **Scenario 3 (Foreign District - LLM still extracts it):**
    * **User Input:** "도쿄 신주쿠구에서 맛있는 라멘집 알려줘"
    * **Expected Output:**
        ```json
        {
            "인원": 1,
            "계절": "여름",
            "점저": "저녁",
            "1인당비용": 15000,
            "업종 중분류": "일식",
            "구": "신주쿠구"
        }
        ```
"""
    
    response = client.chat.completions.create(
        model=your_deployment_name, # 배포된 모델의 이름 (gpt-35-turbo, gpt-4o-mini 등)
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": system_prompt_content # 여기에 정의한 긴 프롬프트 문자열 변수 사용
                    }
                ]
            },
            # 사용자 프롬프트. 실제 사용자 입력 변수를 여기에 넣어 사용하세요.
            {"role": "user", "content": "공덕역 인근 맛집 추천해줘"} 
        ],
        max_tokens=200, # JSON 출력을 위해 max_tokens를 넉넉하게 설정하는 것이 좋습니다.
        # 응답 형식을 JSON으로 명시적으로 지정
        response_format={"type": "json_object"}
    )

    # 4. 응답 출력 및 JSON 파싱
    llm_output_json_str = response.choices[0].message.content
    model_input_data = json.loads(llm_output_json_str)

except Exception as e:
    print(f"API 호출 중 오류 발생: {e}")

    

In [32]:
print(llm_output_json_str)
print(model_input_data)



{
    "인원": 1,
    "계절": "여름",
    "점저": "저녁",
    "1인당비용": 15000,
    "업종 중분류": "한식",
    "구": "마포구"
}
{'인원': 1, '계절': '여름', '점저': '저녁', '1인당비용': 15000, '업종 중분류': '한식', '구': '마포구'}
